<font size="5">Create SNR data set</font>

Create a data set where the test set matches a certain signal-to-noise-ratio (SNR).

In [1]:
# Import modules and dependencies

import os
from os import listdir
from os.path import isdir, join
import pathlib
import shutil

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display
from tensorflow.python.ops import gen_audio_ops as audio_ops


import akida
import akida_models
from akida_models import kws
from akida_models.kws import preprocessing
import cnn2snn
from cnn2snn import check_model_compatibility
from cnn2snn import quantize
from cnn2snn import quantize_layer
from cnn2snn import convert

import librosa
import scipy
from scipy import io
from scipy.io import wavfile
from scipy.io.wavfile import read
import python_speech_features
import random

<font size="5">1. Load the data set</font>

The Google Speech Command data set is loaded in this section of the code as a randomly ordered list of directories. 

In [2]:
# If needed, set seed for experiment reproducibility

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
# Define directories of the data
# Folder Silence only includes the silence category

data_dir = pathlib.Path('data/Modded_Google')
silence_dir = pathlib.Path('data/Modded_Google/silence')

In [4]:
# Check if the keywords are the desired ones

targets = np.array(tf.io.gfile.listdir(str(data_dir)))

targets = targets[targets != 'README.md']

print('Known and unknown commands:', targets)

Known and unknown commands: ['off' 'up' 'down' 'on' 'stop' 'yes' 'right' 'unknown' 'left' 'go' 'no'
 'silence']


In [5]:
# Create list of file names, along with its target 

filenames = []
y = []
for i, target in enumerate(targets):
    print(join(data_dir, target))
    filenames.append(listdir(join(data_dir, target)))
    y.append(np.ones(len(filenames[i])) * i)

data/Modded_Google/off
data/Modded_Google/up
data/Modded_Google/down
data/Modded_Google/on
data/Modded_Google/stop
data/Modded_Google/yes
data/Modded_Google/right
data/Modded_Google/unknown
data/Modded_Google/left
data/Modded_Google/go
data/Modded_Google/no
data/Modded_Google/silence


In [6]:
# Create list with all the silence file names
# The targets in this case are note neccessary because these files are used later as added noise

silence_filenames = []
silence_filenames.append(listdir(silence_dir))
print(np.shape(silence_filenames))

(1, 402)


In [7]:
# Flatten the array to make it a list instead of a list of arrays

filenames = [item for sublist in filenames for item in sublist]
y = [item for sublist in y for item in sublist]
print('Filenames: ', len(filenames))
print('y: ', len(y))

Filenames:  106231
y:  106231


In [8]:
# Exclude files that are not one second in length

filenames_temp = []
y_temp=[]



for index, filename in enumerate(filenames):

    path = join(data_dir, targets[int(y[index])], filename)
    wav_loader = tf.io.read_file(path)
    wav_decoder = tf.audio.decode_wav(wav_loader,desired_channels=1, desired_samples=-1)
    
    if wav_decoder[0].shape == (16000,1):
        filenames_temp.append(filename)
        y_temp.append(y[index])

filenames = filenames_temp
y = y_temp

print('Filenames: ', len(filenames))
print('y: ', len(y))

2022-04-17 20:33:02.300389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-17 20:33:02.358798: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-17 20:33:02.358967: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-17 20:33:02.359617: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Filenames:  95792
y:  95792


In [9]:
# Distribution of all labels after the conversion

unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{0.0: 3427,
 1.0: 3269,
 2.0: 3580,
 3.0: 3471,
 4.0: 3563,
 5.0: 3692,
 6.0: 3448,
 7.0: 60419,
 8.0: 3502,
 9.0: 3478,
 10.0: 3545,
 11.0: 398}

In [10]:
# There are 34975 files that are not unkown or silence 
# Create number that should be equal to 10 % of the final data set
# This number is later used, such that the category "silence" and "unkown" make up 10% of the data.

amount_sil_un = 34975 / 0.8
amount_sil_un = amount_sil_un * 0.1
print(int(amount_sil_un))


4371


In [11]:
# Make file names of silence to one long list 

silence_filenames = np.squeeze(silence_filenames)

# Print the length in order to check if something has changed

print(len(silence_filenames))

402


In [12]:
# Include only the silence files which have a total of one second in length

silence_temp = []


for index, filename in enumerate(silence_filenames):

    path = join(silence_dir, filename)
    wav_loader = tf.io.read_file(path)
    wav_decoder = tf.audio.decode_wav(wav_loader,desired_channels=1, desired_samples=-1)
    
    if wav_decoder[0].shape == (16000,1):
        silence_temp.append(filename)

silence_filenames = silence_temp

print('Silence Filenames: ', len(silence_filenames))

Silence Filenames:  398


In [13]:
# Shuffle file names of the silence category
# Print the first position of the list to check if it worked

print(silence_filenames[0])
random.shuffle(silence_filenames)
print(silence_filenames[0])

31dude_miaowing.wav
12white_noise.wav


In [14]:
# Randomly select files of the "unknown" category in order to match the 10% constraint

filenames_target = list(zip(filenames, y))
random.shuffle(filenames_target)
#print(filenames_target)
counter = 0
index = 0
unknown_id = 7
result = []
for i in filenames_target:
    if filenames_target[index][1] != unknown_id:
        result.append(i)
    if filenames_target[index][1] == unknown_id:
        counter += 1
        if counter <= amount_sil_un:
            result.append(i)
    index += 1
filenames_target = result
    
print(len(filenames_target))
#print(filenames_target)

39744


In [15]:
# Check if the distribution still matches the one from the beginning. 

filenames, y = zip(*filenames_target)
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{0.0: 3427,
 1.0: 3269,
 2.0: 3580,
 3.0: 3471,
 4.0: 3563,
 5.0: 3692,
 6.0: 3448,
 7.0: 4371,
 8.0: 3502,
 9.0: 3478,
 10.0: 3545,
 11.0: 398}

In [16]:
# Duplicate silence files in random order till it is 10% of the data
# The randomness was introduced by previously shuffling the set

counter = 398
silence_id = 11
result = filenames_target
index = 0

while counter <= amount_sil_un:
    for i in filenames_target:
        if filenames_target[index][1] == silence_id:
            if counter <= amount_sil_un:
                result.append(i)
                counter +=1
        index += 1
    
    

filenames_target = result

#print(counter)
#print(len(filenames_target))
#print(filenames_target)

In [17]:
# Shuffle filenames again and unzip

#filenames_target = list(zip(filenames, y))
random.shuffle(filenames_target)
filenames, y = zip(*filenames_target)

# Check if data has the right length

print(len(filenames))

43718


In [18]:
# Check if nothing changed besideds "unknown" and "silence"

unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{0.0: 3427,
 1.0: 3269,
 2.0: 3580,
 3.0: 3471,
 4.0: 3563,
 5.0: 3692,
 6.0: 3448,
 7.0: 4371,
 8.0: 3502,
 9.0: 3478,
 10.0: 3545,
 11.0: 4372}

In [19]:
# Indexing such that no file gets left out due to the integer precision

num_samples = (len(filenames_target))

a = num_samples * 0.8
b = num_samples * 0.1
c = num_samples - int(a) - int(b) - int(b)
int(a)
int(b)
print('80% : ', int(a))
print('10% : ', int(b))
print('Number of total samples: ', num_samples)
print('80% + 10% + 10% : ', int(a) + int(b) + int(b))
print('Number of samples that would be left out : ', int(c))

80% :  34974
10% :  4371
Number of total samples:  43718
80% + 10% + 10% :  43716
Number of samples that would be left out :  2


In [20]:
# Split and print all sets

train_files = filenames[:int(a)+int(c)]
val_files = filenames[int(a)+int(c): int(a)+int(c) + int(b)]
test_files = filenames[-int(b):]

print('Training set size', len(train_files))
print('Validation set size', len(val_files))
print('Test set size', len(test_files))

Training set size 34976
Validation set size 4371
Test set size 4371


In [21]:
# Same splitting procedure for the labels

y_train = y[:int(a)+int(c)]
y_val = y[int(a)+int(c): int(a)+int(c) + int(b)]
y_test = y[-int(b):]

print('Training set size', len(y_train))
print('Validation set size', len(y_val))
print('Test set size', len(y_test))

Training set size 34976
Validation set size 4371
Test set size 4371


<font size="5">2. Define MFCC functions </font>

In this chapter the functions for calculating the MFCCs are defined.

In [22]:
 model_settings = akida_models.kws.preprocessing.prepare_model_settings(sample_rate = 16000, clip_duration_ms = 1000, window_size_ms = 30, window_stride_ms = 10, feature_bin_count = 40)
print(model_settings)

{'desired_samples': 16000, 'window_size_samples': 480, 'window_stride_samples': 160, 'spectrogram_length': 98, 'fingerprint_width': 40, 'fingerprint_size': 3920}


In [27]:
# Function: Create MFCC from given path

def calc_mfcc(path,num_ceps):
    
    # Load wave file
    
    wav_loader = tf.io.read_file(path)
    #wav_decoder = tf.audio.decode_wav(wav_loader,desired_channels=1, desired_samples=model_settings['desired_samples'])
    wav_decoder = tf.audio.decode_wav(wav_loader,desired_channels=1, desired_samples=-1)
    if wav_decoder[0].shape != (16000,1):
        return wav_decoder.audio
        
    
    # Randomly shift position to either 100ms forward or backward 
    
    position = random.randint(0,1)
    if position == 0:
        paddings = [[0, int(model_settings['desired_samples']*0.1)], [0, 0]]
        begin = [int(model_settings['desired_samples']*0.1),0]
    if position == 1:
        paddings = [[int(model_settings['desired_samples']*0.1), 0], [0, 0]]
        begin = [0,0]
    scaled_wav_decoder = tf.multiply(wav_decoder.audio, 1)
    padded_wav_decoder = tf.pad(scaled_wav_decoder, paddings)
    sliced_wav_decoder = tf.slice(padded_wav_decoder, begin = begin, size = [model_settings['desired_samples'], -1])

    # Take random silence file and add it to the signal in range between 0 and 0.1

    rand_silence = random.randint(0,len(silence_filenames) - 1)
    rand_amount = random.uniform(0, 0.1)
    silence_loader = tf.io.read_file(join(silence_dir,silence_filenames[rand_silence]))
    silence_decoder = tf.audio.decode_wav(silence_loader,desired_channels=1, desired_samples=model_settings['desired_samples'])
    silence_padded_wav = tf.add(sliced_wav_decoder, tf.multiply(silence_decoder.audio, rand_amount))
    silence_padded_wav = tf.squeeze(silence_padded_wav)
    silence_padded_wav = silence_padded_wav.numpy()

    # Create MFCC of the augmented signal

    output = librosa.feature.mfcc(y=silence_padded_wav,sr=model_settings['desired_samples'],n_mfcc=model_settings['fingerprint_width'],n_fft=512,win_length=model_settings['window_size_samples'],hop_length=model_settings['window_stride_samples'])



    return output


In [45]:
# Function that creates the MFCC and check if they have the desired length 

def extract_features(samples, labels, num_ceps):
    prob_cnt = 0
    out_x = []
    out_y = []
        
    for index, filename in enumerate(samples):

        # Create path from given filename and target item
        path = join(data_dir, targets[int(labels[index])], 
                    filename)

        # Create MFCCs
        mfccs = calc_mfcc(path,num_ceps)

        if mfccs.shape[1] == num_ceps:
            out_x.append(mfccs)
            out_y.append(labels[index])
            print('Correctly computed :', index)
        else:
            print('Dropped:', index, mfccs.shape)
            prob_cnt += 1

    return out_x, out_y, prob_cnt

In [29]:
# Function for creating noise with the desired SNR
# According to https://medium.com/analytics-vidhya/adding-noise-to-audio-clips-5d8cee24ccb8

def calc_noise_for_SNR(signal,noise,SNR):
    
    #Calculate RMS from signal 
    RMS_s = (np.sqrt(np.mean(np.square(signal)))).astype('float64')
    
    #Calculate RMS from noise
    RMS_n_current = (np.sqrt(np.mean(np.square(noise)))).astype('float64')
    
    #Calculate required RMS from noise 
    RMS_n = (np.sqrt(np.square(RMS_s)/(np.power(10,SNR/10)))).astype('float64')
    
    noise = noise * (RMS_n/RMS_n_current)
    
    return noise

In [36]:
# Function: Create MFCC from given path with the desired SNR

def calc_mfcc_SNR(path,num_ceps,SNR):
    
    # Load wave file
    wav_loader = tf.io.read_file(path)
    #wav_decoder = tf.audio.decode_wav(wav_loader,desired_channels=1, desired_samples=model_settings['desired_samples'])
    wav_decoder = tf.audio.decode_wav(wav_loader,desired_channels=1, desired_samples=-1)
    if wav_decoder[0].shape != (16000,1):
        return wav_decoder.audio
        
    
    # Randomly shift position to either 100ms forward or backward  

        
    position = random.randint(0,1)
    if position == 0:
        paddings = [[0, int(model_settings['desired_samples']*0.1)], [0, 0]]
        begin = [int(model_settings['desired_samples']*0.1),0]
    if position == 1:
        paddings = [[int(model_settings['desired_samples']*0.1), 0], [0, 0]]
        begin = [0,0]
    scaled_wav_decoder = tf.multiply(wav_decoder.audio, 1)
    padded_wav_decoder = tf.pad(scaled_wav_decoder, paddings)
    sliced_wav_decoder = tf.slice(padded_wav_decoder, begin = begin, size = [model_settings['desired_samples'], -1])

    # Add random noise file with a certain ratio (SNR)

    rand_silence = random.randint(0,len(silence_filenames) - 1)
    silence_loader = tf.io.read_file(join(silence_dir,silence_filenames[rand_silence]))
    silence_decoder = tf.audio.decode_wav(silence_loader,desired_channels=1, desired_samples=model_settings['desired_samples'])
    
    # Get the audio data from the decoder 
    # Data has to be float 64 in order to add small amounts of noise, e.g., SNR of 1000
    
    noise = silence_decoder.audio
    noise = np.array(noise,dtype='float64')
    
    # Shape noise in order to match a certain SNR
    
    noise = calc_noise_for_SNR(sliced_wav_decoder,noise,SNR)
    
    
    sliced_wav_decoder = np.array(sliced_wav_decoder,dtype='float64')
    silence_padded_wav = tf.add(sliced_wav_decoder, noise)
    
    # Add the desired noise and check if it worked
    
    power = np.square(sliced_wav_decoder)
    average_power = np.mean(power)
    average_power_db = 10 * np.log10(average_power)
    
    #print('DB: ', average_power_db)
    
    power_noise = np.square(noise)
    average_power_noise = np.mean(power_noise)
    average_power_db_noise = 10 * np.log10(average_power_noise)
    
    print('SNR : ', (average_power_db - average_power_db_noise))
    
    
    silence_padded_wav = tf.squeeze(silence_padded_wav)
    silence_padded_wav = silence_padded_wav.numpy()

    # Create MFCC of the augmented signal 

    output = librosa.feature.mfcc(y=silence_padded_wav,sr=model_settings['desired_samples'],n_mfcc=model_settings['fingerprint_width'],n_fft=512,win_length=model_settings['window_size_samples'],hop_length=model_settings['window_stride_samples'])



    return output

In [37]:
# Function that creates the MFCC and check if they have the desired length and a certain SNR

def extract_features_SNR(samples, labels, num_ceps, SNR):
    prob_cnt = 0
    out_x = []
    out_y = []
        
    for index, filename in enumerate(samples):

        # Create path from given filename and target item
        
        path = join(data_dir, targets[int(labels[index])], 
                    filename)

        # Create MFCCs
        
        mfccs = calc_mfcc_SNR(path,num_ceps,SNR)

        if mfccs.shape[1] == num_ceps:
            out_x.append(mfccs)
            out_y.append(labels[index])
            print('Correctly computed :', index)
        else:
            print('Dropped:', index, mfccs.shape)
            prob_cnt += 1

    return out_x, out_y, prob_cnt

<font size="5">3. Save the MFCCs </font>

Save the MFCCs including their targets seperated into training, validation and test set. Including the test sets with the different SNRs.

In [38]:
# Create test set with SNR of 1
# num_ceps is the desired length of the MFCC

num_ceps = 101
x_test, y_test, prob = extract_features_SNR(test_files, y_test, num_ceps, SNR=1)
print('Removed percentage:', prob / len(y_test))

# Save MFCCs and their target

feature_sets_file = 'test_SNR_1.npz'
np.savez(feature_sets_file, 
         x_test=x_test, 
         y_test=y_test)

SNR :  1.000000079426087
Correctly computed : 0
SNR :  1.00000035783631
Correctly computed : 1
SNR :  1.0000003510736235
Correctly computed : 2
SNR :  1.000000160149746
Correctly computed : 3
SNR :  1.0000002570661977
Correctly computed : 4
SNR :  1.0000001899114466
Correctly computed : 5
SNR :  1.0000005083528762
Correctly computed : 6
SNR :  1.0000004509073506
Correctly computed : 7
SNR :  1.0000000426699316
Correctly computed : 8
SNR :  1.0000004790074257
Correctly computed : 9
SNR :  1.0000007543417233
Correctly computed : 10
SNR :  0.9999994371152923
Correctly computed : 11
SNR :  1.0000003079938011
Correctly computed : 12
SNR :  1.000001059786186
Correctly computed : 13
SNR :  1.000000403236605
Correctly computed : 14
SNR :  1.0000001217727146
Correctly computed : 15
SNR :  1.0000009167458757
Correctly computed : 16
SNR :  0.9999995372932702
Correctly computed : 17
SNR :  1.00000056037835
Correctly computed : 18
SNR :  1.0000001874624758
Correctly computed : 19
SNR :  1.000000304

Correctly computed : 168
SNR :  0.9999999823154937
Correctly computed : 169
SNR :  1.000000706268878
Correctly computed : 170
SNR :  0.9999999861340036
Correctly computed : 171
SNR :  1.000000816467935
Correctly computed : 172
SNR :  0.9999999236593311
Correctly computed : 173
SNR :  0.9999996285306203
Correctly computed : 174
SNR :  1.0000008127955553
Correctly computed : 175
SNR :  0.9999995060297344
Correctly computed : 176
SNR :  1.0000001159007326
Correctly computed : 177
SNR :  1.0000003570681315
Correctly computed : 178
SNR :  1.0000001662843552
Correctly computed : 179
SNR :  1.0000003644904254
Correctly computed : 180
SNR :  0.9999999381338984
Correctly computed : 181
SNR :  0.9999997345737945
Correctly computed : 182
SNR :  1.0000004222706593
Correctly computed : 183
SNR :  1.0000005323193673
Correctly computed : 184
SNR :  1.0000007016854298
Correctly computed : 185
SNR :  0.9999996822713051
Correctly computed : 186
SNR :  1.0000001314829312
Correctly computed : 187
SNR :  1

Correctly computed : 359
SNR :  0.9999999068099399
Correctly computed : 360
SNR :  0.9999999236290869
Correctly computed : 361
SNR :  0.9999996559448334
Correctly computed : 362
SNR :  0.9999999961266042
Correctly computed : 363
SNR :  0.9999998086537865
Correctly computed : 364
SNR :  1.0000003125855699
Correctly computed : 365
SNR :  1.0000003593338391
Correctly computed : 366
SNR :  1.0000003975024185
Correctly computed : 367
SNR :  1.0000005470272555
Correctly computed : 368
SNR :  1.000000086751406
Correctly computed : 369
SNR :  0.999999594644656
Correctly computed : 370
SNR :  1.0000001525392115
Correctly computed : 371
SNR :  1.0000000648615135
Correctly computed : 372
SNR :  0.9999999266656587
Correctly computed : 373
SNR :  1.0000000662167885
Correctly computed : 374
SNR :  0.9999996033462324
Correctly computed : 375
SNR :  1.000000293765254
Correctly computed : 376
SNR :  1.0000000519939505
Correctly computed : 377
SNR :  0.9999997353267851
Correctly computed : 378
SNR :  0.

Correctly computed : 527
SNR :  1.0000000864901608
Correctly computed : 528
SNR :  1.000000445098017
Correctly computed : 529
SNR :  0.9999998153763272
Correctly computed : 530
SNR :  1.0000000901729678
Correctly computed : 531
SNR :  1.0000005252659037
Correctly computed : 532
SNR :  1.0000005044736966
Correctly computed : 533
SNR :  1.0000000108522542
Correctly computed : 534
SNR :  0.9999997236796219
Correctly computed : 535
SNR :  0.999999913372509
Correctly computed : 536
SNR :  1.0000000420193942
Correctly computed : 537
SNR :  1.0000006783402355
Correctly computed : 538
SNR :  1.0000002085624367
Correctly computed : 539
SNR :  1.000000307668408
Correctly computed : 540
SNR :  1.0000004274441388
Correctly computed : 541
SNR :  1.0000001319740264
Correctly computed : 542
SNR :  1.0000000256068127
Correctly computed : 543
SNR :  0.9999998208865328
Correctly computed : 544
SNR :  1.0000000420703223
Correctly computed : 545
SNR :  1.000000158730213
Correctly computed : 546
SNR :  1.0

Correctly computed : 706
SNR :  1.0000006932862355
Correctly computed : 707
SNR :  0.9999994149750648
Correctly computed : 708
SNR :  1.0000004348882428
Correctly computed : 709
SNR :  1.0000001010734323
Correctly computed : 710
SNR :  1.0000000852494075
Correctly computed : 711
SNR :  1.0000002139485176
Correctly computed : 712
SNR :  1.0000001627858524
Correctly computed : 713
SNR :  1.000000511258257
Correctly computed : 714
SNR :  1.0000003669321664
Correctly computed : 715
SNR :  1.0000000041776005
Correctly computed : 716
SNR :  1.00000015282383
Correctly computed : 717
SNR :  0.9999998945659598
Correctly computed : 718
SNR :  1.0000000263517776
Correctly computed : 719
SNR :  1.0000002926662361
Correctly computed : 720
SNR :  1.0000002067381146
Correctly computed : 721
SNR :  0.9999999567790177
Correctly computed : 722
SNR :  1.0000002559246788
Correctly computed : 723
SNR :  0.9999998110486032
Correctly computed : 724
SNR :  1.0000001422464173
Correctly computed : 725
SNR :  0.

...

Correctly computed : 3648
SNR :  0.9999999528104766
Correctly computed : 3649
SNR :  0.9999998810255661
Correctly computed : 3650
SNR :  1.0000001555166218
Correctly computed : 3651
SNR :  1.0000005086801202
Correctly computed : 3652
SNR :  1.0000000721096214
Correctly computed : 3653
SNR :  1.0000003267571813
Correctly computed : 3654
SNR :  1.000000141558644
Correctly computed : 3655
SNR :  0.9999997618064356
Correctly computed : 3656
SNR :  0.9999999682186314
Correctly computed : 3657
SNR :  1.0000004184888525
Correctly computed : 3658
SNR :  0.9999999904970949
Correctly computed : 3659
SNR :  1.0000003308372847
Correctly computed : 3660
SNR :  0.9999995854030388
Correctly computed : 3661
SNR :  1.0000002417556004
Correctly computed : 3662
SNR :  1.0000004005064334
Correctly computed : 3663
SNR :  0.9999994653653133
Correctly computed : 3664
SNR :  1.0000002639493886
Correctly computed : 3665
SNR :  0.9999996564824158
Correctly computed : 3666
SNR :  1.0000001288502318
Correctly com

SNR :  1.0000003888199842
Correctly computed : 3822
SNR :  0.9999996889536362
Correctly computed : 3823
SNR :  1.0000006769841452
Correctly computed : 3824
SNR :  0.999999655986425
Correctly computed : 3825
SNR :  1.0000003153373314
Correctly computed : 3826
SNR :  0.9999999754641209
Correctly computed : 3827
SNR :  1.0000002343715835
Correctly computed : 3828
SNR :  1.0000002135965111
Correctly computed : 3829
SNR :  1.0000003218557936
Correctly computed : 3830
SNR :  1.000000027137574
Correctly computed : 3831
SNR :  1.0000005085371129
Correctly computed : 3832
SNR :  0.9999999102110451
Correctly computed : 3833
SNR :  1.000000077174981
Correctly computed : 3834
SNR :  1.000000468124128
Correctly computed : 3835
SNR :  0.999999512350179
Correctly computed : 3836
SNR :  0.999999825183739
Correctly computed : 3837
SNR :  0.9999999413578919
Correctly computed : 3838
SNR :  0.9999995869748801
Correctly computed : 3839
SNR :  1.0000004490307148
Correctly computed : 3840
SNR :  1.000000288

SNR :  0.9999992311587391
Correctly computed : 3993
SNR :  0.9999994974172388
Correctly computed : 3994
SNR :  0.9999998621829533
Correctly computed : 3995
SNR :  1.000000149698753
Correctly computed : 3996
SNR :  0.9999994486464772
Correctly computed : 3997
SNR :  1.000000570564449
Correctly computed : 3998
SNR :  0.9999996188099942
Correctly computed : 3999
SNR :  1.0000001698016128
Correctly computed : 4000
SNR :  0.9999998605918705
Correctly computed : 4001
SNR :  1.000000422728089
Correctly computed : 4002
SNR :  1.0000004068684554
Correctly computed : 4003
SNR :  1.0000003896922323
Correctly computed : 4004
SNR :  0.9999998461324768
Correctly computed : 4005
SNR :  1.0000002451093621
Correctly computed : 4006
SNR :  1.0000007924946352
Correctly computed : 4007
SNR :  1.0000006704953712
Correctly computed : 4008
SNR :  0.9999998726146302
Correctly computed : 4009
SNR :  0.9999997837922194
Correctly computed : 4010
SNR :  1.0000004549423096
Correctly computed : 4011
SNR :  1.000000

SNR :  1.0000000433267324
Correctly computed : 4156
SNR :  0.9999995781159257
Correctly computed : 4157
SNR :  1.0000003774227544
Correctly computed : 4158
SNR :  0.9999999049924462
Correctly computed : 4159
SNR :  0.9999996422718311
Correctly computed : 4160
SNR :  1.0000000901170765
Correctly computed : 4161
SNR :  1.0000004125359965
Correctly computed : 4162
SNR :  1.000000174777199
Correctly computed : 4163
SNR :  1.000000274290894
Correctly computed : 4164
SNR :  1.0000001366577607
Correctly computed : 4165
SNR :  1.0000008726723237
Correctly computed : 4166
SNR :  0.9999994055245942
Correctly computed : 4167
SNR :  1.000000111395746
Correctly computed : 4168
SNR :  1.0000002964877908
Correctly computed : 4169
SNR :  0.9999996367864199
Correctly computed : 4170
SNR :  1.0000004665513131
Correctly computed : 4171
SNR :  0.9999997338245485
Correctly computed : 4172
SNR :  1.00000052788846
Correctly computed : 4173
SNR :  1.0000002294860337
Correctly computed : 4174
SNR :  1.00000032

SNR :  1.0000003277864842
Correctly computed : 4339
SNR :  1.0000004466905708
Correctly computed : 4340
SNR :  0.9999999895728848
Correctly computed : 4341
SNR :  1.0000003333619034
Correctly computed : 4342
SNR :  1.0000004932604227
Correctly computed : 4343
SNR :  0.9999998208210101
Correctly computed : 4344
SNR :  1.0000001512876402
Correctly computed : 4345
SNR :  1.0000002117203053
Correctly computed : 4346
SNR :  1.0000001239745302
Correctly computed : 4347
SNR :  0.9999997614572038
Correctly computed : 4348
SNR :  1.0000000070984285
Correctly computed : 4349
SNR :  1.000000340178019
Correctly computed : 4350
SNR :  1.0000001335720938
Correctly computed : 4351
SNR :  0.9999998383512381
Correctly computed : 4352
SNR :  0.999999777920074
Correctly computed : 4353
SNR :  1.0000002918647226
Correctly computed : 4354
SNR :  1.0000001227128692
Correctly computed : 4355
SNR :  1.0000005377259313
Correctly computed : 4356
SNR :  0.9999997520433794
Correctly computed : 4357
SNR :  0.99999

In [41]:
# Create test set with SNR of 10
# num_ceps is the desired length of the MFCC

num_ceps = 101
x_test, y_test, prob = extract_features_SNR(test_files, y_test, num_ceps, SNR=10)
print('Removed percentage:', prob / len(y_test))

# Save MFCCs and their target

feature_sets_file = 'test_SNR_10.npz'
np.savez(feature_sets_file, 
         x_test=x_test, 
         y_test=y_test)

SNR :  10.000000079426087
Correctly computed : 0
SNR :  10.000000357836313
Correctly computed : 1
SNR :  9.999999773538686
Correctly computed : 2
SNR :  10.000000372504203
Correctly computed : 3
SNR :  9.999999977646887
Correctly computed : 4
SNR :  10.000000189911447
Correctly computed : 5
SNR :  10.000000508352876
Correctly computed : 6
SNR :  10.00000045090735
Correctly computed : 7
SNR :  10.000000042669924
Correctly computed : 8
SNR :  10.000000479007422
Correctly computed : 9
SNR :  10.00000075434173
Correctly computed : 10
SNR :  10.000000656325739
Correctly computed : 11
SNR :  10.000000388367326
Correctly computed : 12
SNR :  9.99999995756211
Correctly computed : 13
SNR :  10.000000403236598
Correctly computed : 14
SNR :  10.000000121772715
Correctly computed : 15
SNR :  10.000000916745876
Correctly computed : 16
SNR :  9.99999953729327
Correctly computed : 17
SNR :  9.999999949722639
Correctly computed : 18
SNR :  9.999999776781355
Correctly computed : 19
SNR :  10.0000003049

Correctly computed : 187
SNR :  10.000000563977107
Correctly computed : 188
SNR :  10.000000168829928
Correctly computed : 189
SNR :  10.000000117169442
Correctly computed : 190
SNR :  10.000000166795818
Correctly computed : 191
SNR :  9.99999998510998
Correctly computed : 192
SNR :  10.000000474909516
Correctly computed : 193
SNR :  9.999999888729388
Correctly computed : 194
SNR :  9.999999587824927
Correctly computed : 195
SNR :  9.999999661706095
Correctly computed : 196
SNR :  10.000000437309097
Correctly computed : 197
SNR :  9.999999914902018
Correctly computed : 198
SNR :  10.000000189087093
Correctly computed : 199
SNR :  10.000000258206978
Correctly computed : 200
SNR :  10.000000245511323
Correctly computed : 201
SNR :  10.00000009942152
Correctly computed : 202
SNR :  9.999999863040994
Correctly computed : 203
SNR :  9.999999979247235
Correctly computed : 204
SNR :  9.999999936684716
Correctly computed : 205
SNR :  9.999999945490458
Correctly computed : 206
SNR :  10.0000004

Correctly computed : 358
SNR :  10.000000596423241
Correctly computed : 359
SNR :  9.999999892835955
Correctly computed : 360
SNR :  9.99999992362909
Correctly computed : 361
SNR :  9.999999986367271
Correctly computed : 362
SNR :  9.999999996126611
Correctly computed : 363
SNR :  9.999999808653783
Correctly computed : 364
SNR :  10.000000312585577
Correctly computed : 365
SNR :  10.00000035933384
Correctly computed : 366
SNR :  10.000000397502426
Correctly computed : 367
SNR :  10.000000547027263
Correctly computed : 368
SNR :  10.000000509637971
Correctly computed : 369
SNR :  9.999999594644652
Correctly computed : 370
SNR :  10.000000152539211
Correctly computed : 371
SNR :  10.000000149573488
Correctly computed : 372
SNR :  10.000000035860666
Correctly computed : 373
SNR :  10.000000066216785
Correctly computed : 374
SNR :  9.99999960334624
Correctly computed : 375
SNR :  10.000000293765247
Correctly computed : 376
SNR :  10.00000005199395
Correctly computed : 377
SNR :  9.99999973

SNR :  9.999999598594705
Correctly computed : 549
SNR :  10.000000130851717
Correctly computed : 550
SNR :  10.000000147399916
Correctly computed : 551
SNR :  10.000000070298412
Correctly computed : 552
SNR :  10.000000405711567
Correctly computed : 553
SNR :  9.999999971652226
Correctly computed : 554
SNR :  10.000000108827578
Correctly computed : 555
SNR :  10.00000031900661
Correctly computed : 556
SNR :  10.000000506634613
Correctly computed : 557
SNR :  10.000000004785306
Correctly computed : 558
SNR :  10.000000413385425
Correctly computed : 559
SNR :  10.000000135478146
Correctly computed : 560
SNR :  10.00000004809981
Correctly computed : 561
SNR :  10.000000154008163
Correctly computed : 562
SNR :  9.999999774087485
Correctly computed : 563
SNR :  9.999999945100761
Correctly computed : 564
SNR :  9.999999570855714
Correctly computed : 565
SNR :  10.000000224094816
Correctly computed : 566
SNR :  10.00000063339278
Correctly computed : 567
SNR :  9.999999909804181
Correctly comp

SNR :  9.9999998110486
Correctly computed : 724
SNR :  10.000000142246417
Correctly computed : 725
SNR :  9.999999798977022
Correctly computed : 726
SNR :  10.00000003210761
Correctly computed : 727
SNR :  9.999999376536984
Correctly computed : 728
SNR :  9.999999996802352
Correctly computed : 729
SNR :  9.999999784557993
Correctly computed : 730
SNR :  9.999999952049368
Correctly computed : 731
SNR :  10.000000532096717
Correctly computed : 732
SNR :  9.99999974844902
Correctly computed : 733
SNR :  10.000000163532018
Correctly computed : 734
SNR :  10.000000244604955
Correctly computed : 735
SNR :  9.999999680695794
Correctly computed : 736
SNR :  9.999999982962755
Correctly computed : 737
SNR :  10.000000105781318
Correctly computed : 738
SNR :  9.999999784216975
Correctly computed : 739
SNR :  9.999999715585894
Correctly computed : 740
SNR :  10.000000334870592
Correctly computed : 741
SNR :  10.00000019480538
Correctly computed : 742
SNR :  9.99999989455253
Correctly computed : 74

...

SNR :  10.000000531984725
Correctly computed : 3591
SNR :  10.000000876179776
Correctly computed : 3592
SNR :  9.999999713366226
Correctly computed : 3593
SNR :  10.000000605700222
Correctly computed : 3594
SNR :  10.000000277296735
Correctly computed : 3595
SNR :  9.999999732913793
Correctly computed : 3596
SNR :  9.999999679363516
Correctly computed : 3597
SNR :  10.000000078299319
Correctly computed : 3598
SNR :  10.00000002028306
Correctly computed : 3599
SNR :  10.000000038794866
Correctly computed : 3600
SNR :  10.00000010031778
Correctly computed : 3601
SNR :  9.999999977847992
Correctly computed : 3602
SNR :  10.000000054260731
Correctly computed : 3603
SNR :  10.000000280166276
Correctly computed : 3604
SNR :  9.999999723338831
Correctly computed : 3605
SNR :  9.999999747565209
Correctly computed : 3606
SNR :  9.999999917965837
Correctly computed : 3607
SNR :  9.99999997981353
Correctly computed : 3608
SNR :  10.000000765203119
Correctly computed : 3609
SNR :  10.0000003929613

Correctly computed : 3754
SNR :  10.000000306920029
Correctly computed : 3755
SNR :  10.000000440471965
Correctly computed : 3756
SNR :  10.000000117444834
Correctly computed : 3757
SNR :  10.000000406223744
Correctly computed : 3758
SNR :  10.0000002638266
Correctly computed : 3759
SNR :  10.00000016084147
Correctly computed : 3760
SNR :  10.000000219518881
Correctly computed : 3761
SNR :  9.999999799921934
Correctly computed : 3762
SNR :  9.999999343171197
Correctly computed : 3763
SNR :  10.000000278023858
Correctly computed : 3764
SNR :  10.000000405127203
Correctly computed : 3765
SNR :  9.99999985948159
Correctly computed : 3766
SNR :  9.999999557735372
Correctly computed : 3767
SNR :  10.000000652730925
Correctly computed : 3768
SNR :  9.999999780102971
Correctly computed : 3769
SNR :  10.00000046643845
Correctly computed : 3770
SNR :  9.999999629020188
Correctly computed : 3771
SNR :  10.000000482358914
Correctly computed : 3772
SNR :  9.999999846132477
Correctly computed : 377

Correctly computed : 3923
SNR :  9.999999687225255
Correctly computed : 3924
SNR :  10.000000180176468
Correctly computed : 3925
SNR :  9.999999826196198
Correctly computed : 3926
SNR :  9.999999867733493
Correctly computed : 3927
SNR :  10.000000038551292
Correctly computed : 3928
SNR :  10.000000007808453
Correctly computed : 3929
SNR :  9.999999980242418
Correctly computed : 3930
SNR :  9.999999682211076
Correctly computed : 3931
SNR :  9.999999406705406
Correctly computed : 3932
SNR :  10.000000062974138
Correctly computed : 3933
SNR :  9.999999997241542
Correctly computed : 3934
SNR :  10.000000240961072
Correctly computed : 3935
SNR :  9.999999721367843
Correctly computed : 3936
SNR :  10.000000431668404
Correctly computed : 3937
SNR :  9.9999999485509
Correctly computed : 3938
SNR :  10.000000711803708
Correctly computed : 3939
SNR :  10.000000133436501
Correctly computed : 3940
SNR :  10.000000375593288
Correctly computed : 3941
SNR :  10.000000329082212
Correctly computed : 39

Correctly computed : 4092
SNR :  10.000000099025868
Correctly computed : 4093
SNR :  9.999999956557865
Correctly computed : 4094
SNR :  9.999999704626319
Correctly computed : 4095
SNR :  10.000000495189596
Correctly computed : 4096
SNR :  10.00000016345021
Correctly computed : 4097
SNR :  10.00000068195248
Correctly computed : 4098
SNR :  10.000000674951146
Correctly computed : 4099
SNR :  10.000000419174045
Correctly computed : 4100
SNR :  10.000000064382174
Correctly computed : 4101
SNR :  9.999999579710416
Correctly computed : 4102
SNR :  10.000000112407335
Correctly computed : 4103
SNR :  9.999999498558275
Correctly computed : 4104
SNR :  10.000000437611332
Correctly computed : 4105
SNR :  9.999999943594894
Correctly computed : 4106
SNR :  10.00000016023818
Correctly computed : 4107
SNR :  10.000000243481399
Correctly computed : 4108
SNR :  9.999999825787874
Correctly computed : 4109
SNR :  10.000000131360355
Correctly computed : 4110
SNR :  9.999999964431836
Correctly computed : 4

Correctly computed : 4264
SNR :  10.000000964434264
Correctly computed : 4265
SNR :  10.000000388825157
Correctly computed : 4266
SNR :  9.999999271051294
Correctly computed : 4267
SNR :  9.99999964873231
Correctly computed : 4268
SNR :  10.000000041699934
Correctly computed : 4269
SNR :  10.000000222312451
Correctly computed : 4270
SNR :  9.999999786148049
Correctly computed : 4271
SNR :  9.999999939962706
Correctly computed : 4272
SNR :  10.000000272106742
Correctly computed : 4273
SNR :  10.00000039969926
Correctly computed : 4274
SNR :  9.999999989391547
Correctly computed : 4275
SNR :  9.99999981640483
Correctly computed : 4276
SNR :  10.000000575831432
Correctly computed : 4277
SNR :  9.999999505194804
Correctly computed : 4278
SNR :  10.000000445426338
Correctly computed : 4279
SNR :  10.00000032411662
Correctly computed : 4280
SNR :  10.000000010762037
Correctly computed : 4281
SNR :  10.0000000234931
Correctly computed : 4282
SNR :  10.000000104845281
Correctly computed : 4283

In [42]:
# Create test set with SNR of 20
# num_ceps is the desired length of the MFCC

num_ceps = 101
x_test, y_test, prob = extract_features_SNR(test_files, y_test, num_ceps, SNR=20)
print('Removed percentage:', prob / len(y_test))

# Save MFCCs and their target

feature_sets_file = 'test_SNR_20.npz'
np.savez(feature_sets_file, 
         x_test=x_test, 
         y_test=y_test)

SNR :  20.000000079426087
Correctly computed : 0
SNR :  20.00000035783631
Correctly computed : 1
SNR :  19.999999773538686
Correctly computed : 2
SNR :  20.000000372504203
Correctly computed : 3
SNR :  19.999999977646887
Correctly computed : 4
SNR :  20.000000189911443
Correctly computed : 5
SNR :  20.000000508352876
Correctly computed : 6
SNR :  20.00000045090735
Correctly computed : 7
SNR :  20.000000042669924
Correctly computed : 8
SNR :  19.999999946495443
Correctly computed : 9
SNR :  20.00000075434173
Correctly computed : 10
SNR :  20.00000065632574
Correctly computed : 11
SNR :  20.000000388367326
Correctly computed : 12
SNR :  19.99999995756211
Correctly computed : 13
SNR :  20.000000403236605
Correctly computed : 14
SNR :  20.000000121772715
Correctly computed : 15
SNR :  20.000000054645596
Correctly computed : 16
SNR :  19.99999953729327
Correctly computed : 17
SNR :  19.99999994972264
Correctly computed : 18
SNR :  20.00000018746248
Correctly computed : 19
SNR :  20.00000030

Correctly computed : 171
SNR :  20.000000395495746
Correctly computed : 172
SNR :  19.99999992365933
Correctly computed : 173
SNR :  19.999999628530617
Correctly computed : 174
SNR :  20.000000812795555
Correctly computed : 175
SNR :  19.99999950602973
Correctly computed : 176
SNR :  20.00000011590074
Correctly computed : 177
SNR :  20.00000035706813
Correctly computed : 178
SNR :  20.000000176423313
Correctly computed : 179
SNR :  19.999999827605503
Correctly computed : 180
SNR :  19.999999360300272
Correctly computed : 181
SNR :  19.999999734573795
Correctly computed : 182
SNR :  20.00000042227066
Correctly computed : 183
SNR :  20.00000009511193
Correctly computed : 184
SNR :  20.00000070168543
Correctly computed : 185
SNR :  19.999999682271305
Correctly computed : 186
SNR :  20.00000013148294
Correctly computed : 187
SNR :  20.000000563977107
Correctly computed : 188
SNR :  20.000000143352025
Correctly computed : 189
SNR :  20.00000064794622
Correctly computed : 190
SNR :  20.00000

Correctly computed : 349
SNR :  19.999999975528237
Correctly computed : 350
SNR :  20.000000053635095
Correctly computed : 351
SNR :  19.999999631678364
Correctly computed : 352
SNR :  19.99999959402875
Correctly computed : 353
SNR :  20.00000057748087
Correctly computed : 354
SNR :  19.999999440143096
Correctly computed : 355
SNR :  19.999999945136317
Correctly computed : 356
SNR :  20.00000032308101
Correctly computed : 357
SNR :  20.000000126170924
Correctly computed : 358
SNR :  20.00000031937803
Correctly computed : 359
SNR :  19.99999990680994
Correctly computed : 360
SNR :  19.999999923629083
Correctly computed : 361
SNR :  19.99999965594484
Correctly computed : 362
SNR :  19.999999996126604
Correctly computed : 363
SNR :  20.000000023194605
Correctly computed : 364
SNR :  20.000000312585577
Correctly computed : 365
SNR :  20.00000048966248
Correctly computed : 366
SNR :  20.000000397502426
Correctly computed : 367
SNR :  20.000000547027263
Correctly computed : 368
SNR :  20.000

Correctly computed : 521
SNR :  19.999999460432704
Correctly computed : 522
SNR :  19.99999941187837
Correctly computed : 523
SNR :  19.99999941187837
Correctly computed : 524
SNR :  20.000000559148948
Correctly computed : 525
SNR :  19.999999740313207
Correctly computed : 526
SNR :  20.00000037841906
Correctly computed : 527
SNR :  19.99999973795628
Correctly computed : 528
SNR :  20.000000445098014
Correctly computed : 529
SNR :  19.99999981537632
Correctly computed : 530
SNR :  20.000000141435102
Correctly computed : 531
SNR :  19.99999964202325
Correctly computed : 532
SNR :  20.000000090289902
Correctly computed : 533
SNR :  19.99999971103409
Correctly computed : 534
SNR :  20.000000161693546
Correctly computed : 535
SNR :  20.000000558255376
Correctly computed : 536
SNR :  20.00000004201939
Correctly computed : 537
SNR :  20.000000678340243
Correctly computed : 538
SNR :  20.000000640852996
Correctly computed : 539
SNR :  20.000000307668408
Correctly computed : 540
SNR :  19.9999

Correctly computed : 712
SNR :  20.00000016278585
Correctly computed : 713
SNR :  20.000000596845112
Correctly computed : 714
SNR :  20.00000060402113
Correctly computed : 715
SNR :  20.00000034610685
Correctly computed : 716
SNR :  20.000000152823837
Correctly computed : 717
SNR :  20.00000020767512
Correctly computed : 718
SNR :  20.00000018979744
Correctly computed : 719
SNR :  20.000000367642574
Correctly computed : 720
SNR :  20.00000000482802
Correctly computed : 721
SNR :  19.999999956779014
Correctly computed : 722
SNR :  20.000000255924675
Correctly computed : 723
SNR :  19.999999782858314
Correctly computed : 724
SNR :  19.99999963246725
Correctly computed : 725
SNR :  19.999999706008417
Correctly computed : 726
SNR :  20.00000042665743
Correctly computed : 727
SNR :  19.999999376536984
Correctly computed : 728
SNR :  19.99999998681751
Correctly computed : 729
SNR :  20.000000369999544
Correctly computed : 730
SNR :  19.999999952049368
Correctly computed : 731
SNR :  19.99999

...

Correctly computed : 3636
SNR :  19.99999982021826
Correctly computed : 3637
SNR :  19.999999796831073
Correctly computed : 3638
SNR :  20.00000025869614
Correctly computed : 3639
SNR :  20.000000094592913
Correctly computed : 3640
SNR :  20.00000005833666
Correctly computed : 3641
SNR :  19.99999968379492
Correctly computed : 3642
SNR :  20.00000009174821
Correctly computed : 3643
SNR :  19.99999943770322
Correctly computed : 3644
SNR :  19.999999790709406
Correctly computed : 3645
SNR :  20.00000001048486
Correctly computed : 3646
SNR :  19.999999710634114
Correctly computed : 3647
SNR :  20.00000028229051
Correctly computed : 3648
SNR :  19.999999952810477
Correctly computed : 3649
SNR :  20.00000033002255
Correctly computed : 3650
SNR :  20.000000281320645
Correctly computed : 3651
SNR :  20.000000508680117
Correctly computed : 3652
SNR :  20.000000314765174
Correctly computed : 3653
SNR :  19.9999999557051
Correctly computed : 3654
SNR :  20.00000031701162
Correctly computed : 365

SNR :  20.000000404478016
Correctly computed : 3811
SNR :  19.999999861371336
Correctly computed : 3812
SNR :  19.99999991575458
Correctly computed : 3813
SNR :  20.00000006455024
Correctly computed : 3814
SNR :  20.000000269578003
Correctly computed : 3815
SNR :  19.999999781509676
Correctly computed : 3816
SNR :  19.99999998641221
Correctly computed : 3817
SNR :  20.00000014669586
Correctly computed : 3818
SNR :  20.000000294340836
Correctly computed : 3819
SNR :  19.99999984136684
Correctly computed : 3820
SNR :  20.000000369049367
Correctly computed : 3821
SNR :  20.000000388819977
Correctly computed : 3822
SNR :  20.000000060019737
Correctly computed : 3823
SNR :  20.000000676984143
Correctly computed : 3824
SNR :  20.00000002623858
Correctly computed : 3825
SNR :  20.000000215253813
Correctly computed : 3826
SNR :  19.99999997546412
Correctly computed : 3827
SNR :  20.00000018980596
Correctly computed : 3828
SNR :  20.000000317136603
Correctly computed : 3829
SNR :  20.0000003218

Correctly computed : 3984
SNR :  19.99999962130201
Correctly computed : 3985
SNR :  20.000000099633823
Correctly computed : 3986
SNR :  20.000000616545528
Correctly computed : 3987
SNR :  20.000000124482213
Correctly computed : 3988
SNR :  19.9999999420588
Correctly computed : 3989
SNR :  20.000000664094642
Correctly computed : 3990
SNR :  19.99999952044483
Correctly computed : 3991
SNR :  20.00000006993899
Correctly computed : 3992
SNR :  19.999999528013927
Correctly computed : 3993
SNR :  20.000000594710567
Correctly computed : 3994
SNR :  20.00000031747921
Correctly computed : 3995
SNR :  20.00000014969875
Correctly computed : 3996
SNR :  19.99999989016608
Correctly computed : 3997
SNR :  19.999999755870935
Correctly computed : 3998
SNR :  19.99999961881
Correctly computed : 3999
SNR :  19.999999961853185
Correctly computed : 4000
SNR :  20.00000056438656
Correctly computed : 4001
SNR :  19.999999964294172
Correctly computed : 4002
SNR :  20.000000406868455
Correctly computed : 4003

Correctly computed : 4159
SNR :  20.000000212596184
Correctly computed : 4160
SNR :  20.00000046698693
Correctly computed : 4161
SNR :  20.000000412535993
Correctly computed : 4162
SNR :  20.00000062923995
Correctly computed : 4163
SNR :  20.000000274290887
Correctly computed : 4164
SNR :  20.000000136657757
Correctly computed : 4165
SNR :  20.000000419816935
Correctly computed : 4166
SNR :  19.99999940552459
Correctly computed : 4167
SNR :  20.00000011139575
Correctly computed : 4168
SNR :  20.00000029648779
Correctly computed : 4169
SNR :  20.000000023210458
Correctly computed : 4170
SNR :  20.000000466551306
Correctly computed : 4171
SNR :  19.999999733824552
Correctly computed : 4172
SNR :  19.999999777453816
Correctly computed : 4173
SNR :  19.999999858322433
Correctly computed : 4174
SNR :  20.00000049073711
Correctly computed : 4175
SNR :  20.000000641565112
Correctly computed : 4176
SNR :  20.000000317752114
Correctly computed : 4177
SNR :  20.000000387779156
Correctly computed

Correctly computed : 4334
SNR :  19.999999656969806
Correctly computed : 4335
SNR :  20.000000674715537
Correctly computed : 4336
SNR :  19.999999707810545
Correctly computed : 4337
SNR :  20.0000001393868
Correctly computed : 4338
SNR :  20.000000377103177
Correctly computed : 4339
SNR :  20.000000446690578
Correctly computed : 4340
SNR :  19.99999998957288
Correctly computed : 4341
SNR :  19.99999997655759
Correctly computed : 4342
SNR :  20.00000049326042
Correctly computed : 4343
SNR :  20.000000198895712
Correctly computed : 4344
SNR :  20.00000031704085
Correctly computed : 4345
SNR :  19.999999860052725
Correctly computed : 4346
SNR :  20.000000123974527
Correctly computed : 4347
SNR :  20.000000296644266
Correctly computed : 4348
SNR :  20.00000000709843
Correctly computed : 4349
SNR :  20.00000034017802
Correctly computed : 4350
SNR :  20.0000001335721
Correctly computed : 4351
SNR :  19.999999838351236
Correctly computed : 4352
SNR :  19.999999777920074
Correctly computed : 4

In [43]:
# Create test set with SNR of 100
# num_ceps is the desired length of the MFCC

num_ceps = 101
x_test, y_test, prob = extract_features_SNR(test_files, y_test, num_ceps, SNR=100)
print('Removed percentage:', prob / len(y_test))

# Save MFCCs and their target

feature_sets_file = 'test_SNR_100.npz'
np.savez(feature_sets_file, 
         x_test=x_test, 
         y_test=y_test)

SNR :  100.00000007942609
Correctly computed : 0
SNR :  100.00000067068379
Correctly computed : 1
SNR :  99.99999977353868
Correctly computed : 2
SNR :  100.0000003725042
Correctly computed : 3
SNR :  100.00000025706618
Correctly computed : 4
SNR :  100.00000018991145
Correctly computed : 5
SNR :  99.99999997796921
Correctly computed : 6
SNR :  100.00000021758419
Correctly computed : 7
SNR :  99.99999985623023
Correctly computed : 8
SNR :  99.99999994649545
Correctly computed : 9
SNR :  100.00000075434171
Correctly computed : 10
SNR :  100.00000065632574
Correctly computed : 11
SNR :  100.00000038836734
Correctly computed : 12
SNR :  99.99999995756211
Correctly computed : 13
SNR :  99.99999971190435
Correctly computed : 14
SNR :  100.0000001217727
Correctly computed : 15
SNR :  100.00000005464558
Correctly computed : 16
SNR :  99.99999995464036
Correctly computed : 17
SNR :  100.00000056037835
Correctly computed : 18
SNR :  99.99999977678137
Correctly computed : 19
SNR :  100.000000030

Correctly computed : 179
SNR :  100.00000036449043
Correctly computed : 180
SNR :  99.99999936030028
Correctly computed : 181
SNR :  99.99999973457379
Correctly computed : 182
SNR :  100.00000042227066
Correctly computed : 183
SNR :  100.00000053231938
Correctly computed : 184
SNR :  100.00000041274495
Correctly computed : 185
SNR :  99.9999996822713
Correctly computed : 186
SNR :  100.00000013148293
Correctly computed : 187
SNR :  100.00000056397711
Correctly computed : 188
SNR :  100.00000016882993
Correctly computed : 189
SNR :  100.00000011716944
Correctly computed : 190
SNR :  99.99999963126577
Correctly computed : 191
SNR :  99.99999993901936
Correctly computed : 192
SNR :  100.00000046906517
Correctly computed : 193
SNR :  99.99999988872939
Correctly computed : 194
SNR :  100.00000001523281
Correctly computed : 195
SNR :  100.00000153023014
Correctly computed : 196
SNR :  100.0000004373091
Correctly computed : 197
SNR :  99.9999999258422
Correctly computed : 198
SNR :  99.999999

SNR :  99.99999999978306
Correctly computed : 348
SNR :  100.00000015998236
Correctly computed : 349
SNR :  99.99999997074275
Correctly computed : 350
SNR :  99.9999995579438
Correctly computed : 351
SNR :  100.00000073712147
Correctly computed : 352
SNR :  99.99999959402874
Correctly computed : 353
SNR :  100.00000007879763
Correctly computed : 354
SNR :  100.00000028597813
Correctly computed : 355
SNR :  99.99999962866981
Correctly computed : 356
SNR :  100.00000010660506
Correctly computed : 357
SNR :  100.0000001261709
Correctly computed : 358
SNR :  100.00000059642323
Correctly computed : 359
SNR :  99.99999990680993
Correctly computed : 360
SNR :  100.00000012112454
Correctly computed : 361
SNR :  99.99999965594483
Correctly computed : 362
SNR :  99.9999999961266
Correctly computed : 363
SNR :  99.99999980865377
Correctly computed : 364
SNR :  100.00000031699659
Correctly computed : 365
SNR :  100.00000035933384
Correctly computed : 366
SNR :  100.00000039750242
Correctly compute

Correctly computed : 539
SNR :  100.00000030766842
Correctly computed : 540
SNR :  100.00000042744414
Correctly computed : 541
SNR :  100.00000013197405
Correctly computed : 542
SNR :  100.00000002560681
Correctly computed : 543
SNR :  100.00000004162172
Correctly computed : 544
SNR :  100.00000004207033
Correctly computed : 545
SNR :  100.00000015873022
Correctly computed : 546
SNR :  100.00000017630893
Correctly computed : 547
SNR :  100.00000056206291
Correctly computed : 548
SNR :  99.99999959859471
Correctly computed : 549
SNR :  99.99999996508694
Correctly computed : 550
SNR :  100.00000014739993
Correctly computed : 551
SNR :  99.9999997382404
Correctly computed : 552
SNR :  100.00000033394596
Correctly computed : 553
SNR :  99.99999997165223
Correctly computed : 554
SNR :  99.99999998263743
Correctly computed : 555
SNR :  100.0000003190066
Correctly computed : 556
SNR :  100.00000028245623
Correctly computed : 557
SNR :  99.99999937249875
Correctly computed : 558
SNR :  100.000

Correctly computed : 713
SNR :  100.00000051125824
Correctly computed : 714
SNR :  100.00000036693217
Correctly computed : 715
SNR :  100.0000000041776
Correctly computed : 716
SNR :  100.00000015282384
Correctly computed : 717
SNR :  100.00000020767513
Correctly computed : 718
SNR :  100.00000018979745
Correctly computed : 719
SNR :  100.00000036764257
Correctly computed : 720
SNR :  100.00000020673812
Correctly computed : 721
SNR :  99.99999995677902
Correctly computed : 722
SNR :  100.00000013603727
Correctly computed : 723
SNR :  99.99999978285831
Correctly computed : 724
SNR :  100.0000001422464
Correctly computed : 725
SNR :  99.99999979897702
Correctly computed : 726
SNR :  100.00000003210762
Correctly computed : 727
SNR :  99.9999997135489
Correctly computed : 728
SNR :  99.99999999680236
Correctly computed : 729
SNR :  100.00000036999954
Correctly computed : 730
SNR :  99.99999962877786
Correctly computed : 731
SNR :  99.99999998097826
Correctly computed : 732
SNR :  100.00000

...

SNR :  100.0000003494072
Correctly computed : 3624
SNR :  100.00000003332696
Correctly computed : 3625
SNR :  100.00000015092176
Correctly computed : 3626
SNR :  100.0000005535071
Correctly computed : 3627
SNR :  99.99999990080941
Correctly computed : 3628
SNR :  99.9999996796956
Correctly computed : 3629
SNR :  100.00000119808722
Correctly computed : 3630
SNR :  99.99999986152801
Correctly computed : 3631
SNR :  100.00000035031597
Correctly computed : 3632
SNR :  100.00000003920374
Correctly computed : 3633
SNR :  99.99999993655706
Correctly computed : 3634
SNR :  99.99999996060751
Correctly computed : 3635
SNR :  100.00000023745325
Correctly computed : 3636
SNR :  99.99999995296557
Correctly computed : 3637
SNR :  99.99999979683108
Correctly computed : 3638
SNR :  99.99999951208792
Correctly computed : 3639
SNR :  100.00000045048364
Correctly computed : 3640
SNR :  99.99999953919627
Correctly computed : 3641
SNR :  99.99999968379493
Correctly computed : 3642
SNR :  100.0000000917482


SNR :  99.99999980670407
Correctly computed : 3796
SNR :  100.00000007247874
Correctly computed : 3797
SNR :  100.00000072595896
Correctly computed : 3798
SNR :  100.00000007098188
Correctly computed : 3799
SNR :  100.00000004213092
Correctly computed : 3800
SNR :  100.00000037902205
Correctly computed : 3801
SNR :  100.00000024027894
Correctly computed : 3802
SNR :  99.99999947972213
Correctly computed : 3803
SNR :  99.999999946891
Correctly computed : 3804
SNR :  99.99999972333883
Correctly computed : 3805
SNR :  100.00000093475654
Correctly computed : 3806
SNR :  100.00000028197735
Correctly computed : 3807
SNR :  100.00000005275069
Correctly computed : 3808
SNR :  100.00000040564814
Correctly computed : 3809
SNR :  99.99999991824842
Correctly computed : 3810
SNR :  99.99999986720557
Correctly computed : 3811
SNR :  99.99999986137135
Correctly computed : 3812
SNR :  99.99999991575457
Correctly computed : 3813
SNR :  100.00000022012672
Correctly computed : 3814
SNR :  100.00000026957

SNR :  100.00000041478638
Correctly computed : 3970
SNR :  99.99999947476152
Correctly computed : 3971
SNR :  99.99999970883974
Correctly computed : 3972
SNR :  100.00000007147366
Correctly computed : 3973
SNR :  99.99999956855264
Correctly computed : 3974
SNR :  100.00000025062778
Correctly computed : 3975
SNR :  100.00000066495569
Correctly computed : 3976
SNR :  100.00000012756033
Correctly computed : 3977
SNR :  100.0000000170962
Correctly computed : 3978
SNR :  99.9999996537469
Correctly computed : 3979
SNR :  100.00000010934748
Correctly computed : 3980
SNR :  100.00000002159784
Correctly computed : 3981
SNR :  100.00000020608964
Correctly computed : 3982
SNR :  100.00000034062582
Correctly computed : 3983
SNR :  99.99999993262114
Correctly computed : 3984
SNR :  99.99999992664821
Correctly computed : 3985
SNR :  100.00000009963381
Correctly computed : 3986
SNR :  100.00000061654552
Correctly computed : 3987
SNR :  99.99999969500591
Correctly computed : 3988
SNR :  99.99999994205

SNR :  100.00000051631787
Correctly computed : 4142
SNR :  99.99999962621126
Correctly computed : 4143
SNR :  100.00000038876306
Correctly computed : 4144
SNR :  100.00000001281171
Correctly computed : 4145
SNR :  99.99999991115021
Correctly computed : 4146
SNR :  100.00000009456807
Correctly computed : 4147
SNR :  100.00000084767603
Correctly computed : 4148
SNR :  100.00000040039401
Correctly computed : 4149
SNR :  99.99999980246307
Correctly computed : 4150
SNR :  100.00000067765659
Correctly computed : 4151
SNR :  100.00000074791475
Correctly computed : 4152
SNR :  99.99999978882526
Correctly computed : 4153
SNR :  100.00000027593663
Correctly computed : 4154
SNR :  100.00000072859098
Correctly computed : 4155
SNR :  100.00000004332672
Correctly computed : 4156
SNR :  99.99999957811592
Correctly computed : 4157
SNR :  100.00000037742277
Correctly computed : 4158
SNR :  99.9999997171213
Correctly computed : 4159
SNR :  100.00000021259618
Correctly computed : 4160
SNR :  100.00000046

Correctly computed : 4313
SNR :  99.9999993636555
Correctly computed : 4314
SNR :  100.00000003459564
Correctly computed : 4315
SNR :  99.99999980399589
Correctly computed : 4316
SNR :  100.00000050105143
Correctly computed : 4317
SNR :  100.00000015220526
Correctly computed : 4318
SNR :  99.99999961064887
Correctly computed : 4319
SNR :  99.99999998302138
Correctly computed : 4320
SNR :  99.99999941484944
Correctly computed : 4321
SNR :  99.99999963880747
Correctly computed : 4322
SNR :  100.00000005475118
Correctly computed : 4323
SNR :  100.0000004126307
Correctly computed : 4324
SNR :  100.00000076355312
Correctly computed : 4325
SNR :  100.00000049045586
Correctly computed : 4326
SNR :  99.99999971150619
Correctly computed : 4327
SNR :  100.00000007253786
Correctly computed : 4328
SNR :  99.99999999170399
Correctly computed : 4329
SNR :  100.00000015924557
Correctly computed : 4330
SNR :  99.99999994605855
Correctly computed : 4331
SNR :  99.99999978345448
Correctly computed : 433

In [44]:
# Create test set with SNR of 1000
# num_ceps is the desired length of the MFCC

num_ceps = 101
x_test, y_test, prob = extract_features_SNR(test_files, y_test, num_ceps, SNR=1000)
print('Removed percentage:', prob / len(y_test))

# Save MFCCs and their target

feature_sets_file = 'test_SNR_1000.npz'
np.savez(feature_sets_file, 
         x_test=x_test, 
         y_test=y_test)

SNR :  1000.0000000794261
Correctly computed : 0
SNR :  1000.0000003578364
Correctly computed : 1
SNR :  1000.0000003510737
Correctly computed : 2
SNR :  1000.0000003725042
Correctly computed : 3
SNR :  999.9999999776469
Correctly computed : 4
SNR :  999.9999999835776
Correctly computed : 5
SNR :  1000.0000005083529
Correctly computed : 6
SNR :  1000.0000004509074
Correctly computed : 7
SNR :  999.9999998562303
Correctly computed : 8
SNR :  999.9999999464956
Correctly computed : 9
SNR :  999.9999997201348
Correctly computed : 10
SNR :  1000.0000006563257
Correctly computed : 11
SNR :  1000.0000003883673
Correctly computed : 12
SNR :  1000.0000010597862
Correctly computed : 13
SNR :  1000.0000004032366
Correctly computed : 14
SNR :  1000.0000001217727
Correctly computed : 15
SNR :  1000.0000009167459
Correctly computed : 16
SNR :  999.9999999546403
Correctly computed : 17
SNR :  999.9999999497227
Correctly computed : 18
SNR :  999.9999997767814
Correctly computed : 19
SNR :  1000.000000

SNR :  1000.0000003839533
Correctly computed : 169
SNR :  1000.0000007062688
Correctly computed : 170
SNR :  1000.0000005364301
Correctly computed : 171
SNR :  1000.0000008164681
Correctly computed : 172
SNR :  999.9999999236594
Correctly computed : 173
SNR :  999.9999996285306
Correctly computed : 174
SNR :  999.9999998563873
Correctly computed : 175
SNR :  999.9999995060298
Correctly computed : 176
SNR :  1000.0000001159008
Correctly computed : 177
SNR :  1000.0000003570682
Correctly computed : 178
SNR :  1000.0000001764232
Correctly computed : 179
SNR :  1000.0000003644905
Correctly computed : 180
SNR :  999.9999993603003
Correctly computed : 181
SNR :  999.9999997345737
Correctly computed : 182
SNR :  999.9999997457207
Correctly computed : 183
SNR :  1000.0000005323194
Correctly computed : 184
SNR :  1000.000000412745
Correctly computed : 185
SNR :  999.9999996822713
Correctly computed : 186
SNR :  1000.0000001314828
Correctly computed : 187
SNR :  1000.0000003707356
Correctly comp

Correctly computed : 339
SNR :  999.9999996530063
Correctly computed : 340
SNR :  999.9999998167932
Correctly computed : 341
SNR :  1000.0000001136129
Correctly computed : 342
SNR :  1000.0000002456687
Correctly computed : 343
SNR :  999.9999998352039
Correctly computed : 344
SNR :  1000.0000003693059
Correctly computed : 345
SNR :  999.999999486775
Correctly computed : 346
SNR :  999.9999997524502
Correctly computed : 347
SNR :  999.9999995274035
Correctly computed : 348
SNR :  1000.0000004814374
Correctly computed : 349
SNR :  999.9999999755282
Correctly computed : 350
SNR :  999.9999995579439
Correctly computed : 351
SNR :  1000.0000007371215
Correctly computed : 352
SNR :  999.9999995940287
Correctly computed : 353
SNR :  1000.0000005774809
Correctly computed : 354
SNR :  1000.000000285978
Correctly computed : 355
SNR :  999.9999996286697
Correctly computed : 356
SNR :  1000.000000106605
Correctly computed : 357
SNR :  999.9999993853759
Correctly computed : 358
SNR :  1000.00000059

Correctly computed : 512
SNR :  999.9999994257621
Correctly computed : 513
SNR :  999.999999623294
Correctly computed : 514
SNR :  999.9999994922686
Correctly computed : 515
SNR :  999.9999996771468
Correctly computed : 516
SNR :  1000.0000001360163
Correctly computed : 517
SNR :  999.9999999184136
Correctly computed : 518
SNR :  1000.000000639914
Correctly computed : 519
SNR :  1000.0000003143036
Correctly computed : 520
SNR :  1000.0000000308735
Correctly computed : 521
SNR :  1000.000000513545
Correctly computed : 522
SNR :  1000.0000002148088
Correctly computed : 523
SNR :  999.9999994118784
Correctly computed : 524
SNR :  1000.0000009347566
Correctly computed : 525
SNR :  999.9999997403133
Correctly computed : 526
SNR :  1000.0000003784193
Correctly computed : 527
SNR :  1000.0000000864902
Correctly computed : 528
SNR :  1000.0000004450981
Correctly computed : 529
SNR :  1000.000000133466
Correctly computed : 530
SNR :  1000.0000000901731
Correctly computed : 531
SNR :  999.999999

SNR :  1000.0000006561041
Correctly computed : 702
SNR :  999.9999997566973
Correctly computed : 703
SNR :  1000.0000002229798
Correctly computed : 704
SNR :  999.9999999166798
Correctly computed : 705
SNR :  1000.0000000879503
Correctly computed : 706
SNR :  999.9999995189555
Correctly computed : 707
SNR :  999.999999414975
Correctly computed : 708
SNR :  1000.0000004348882
Correctly computed : 709
SNR :  999.9999998702203
Correctly computed : 710
SNR :  1000.0000000852494
Correctly computed : 711
SNR :  1000.0000002139486
Correctly computed : 712
SNR :  999.9999998827656
Correctly computed : 713
SNR :  1000.0000005112581
Correctly computed : 714
SNR :  1000.000000604021
Correctly computed : 715
SNR :  1000.000000346107
Correctly computed : 716
SNR :  1000.0000002867963
Correctly computed : 717
SNR :  999.9999998945659
Correctly computed : 718
SNR :  1000.0000000263518
Correctly computed : 719
SNR :  1000.0000003676424
Correctly computed : 720
SNR :  1000.000000206738
Correctly comput

...

Correctly computed : 3656
SNR :  999.9999998084807
Correctly computed : 3657
SNR :  1000.0000004184889
Correctly computed : 3658
SNR :  999.999999990497
Correctly computed : 3659
SNR :  999.9999999786271
Correctly computed : 3660
SNR :  1000.0000007004884
Correctly computed : 3661
SNR :  1000.0000001621737
Correctly computed : 3662
SNR :  1000.0000001514114
Correctly computed : 3663
SNR :  999.9999999579865
Correctly computed : 3664
SNR :  1000.0000002639495
Correctly computed : 3665
SNR :  999.9999996564825
Correctly computed : 3666
SNR :  1000.0000001288502
Correctly computed : 3667
SNR :  999.9999997097558
Correctly computed : 3668
SNR :  1000.0000005687559
Correctly computed : 3669
SNR :  999.9999999782274
Correctly computed : 3670
SNR :  1000.000000057351
Correctly computed : 3671
SNR :  999.9999996116503
Correctly computed : 3672
SNR :  999.9999999017393
Correctly computed : 3673
SNR :  1000.0000005531158
Correctly computed : 3674
SNR :  999.9999997477839
Correctly computed : 367

Correctly computed : 3827
SNR :  1000.0000002343716
Correctly computed : 3828
SNR :  1000.0000003171365
Correctly computed : 3829
SNR :  1000.000000258101
Correctly computed : 3830
SNR :  1000.0000005867606
Correctly computed : 3831
SNR :  999.9999998687717
Correctly computed : 3832
SNR :  999.999999910211
Correctly computed : 3833
SNR :  1000.000000077175
Correctly computed : 3834
SNR :  1000.0000005605859
Correctly computed : 3835
SNR :  999.9999998869345
Correctly computed : 3836
SNR :  999.9999998251839
Correctly computed : 3837
SNR :  999.9999993718602
Correctly computed : 3838
SNR :  1000.0000002517222
Correctly computed : 3839
SNR :  1000.0000004490308
Correctly computed : 3840
SNR :  1000.0000001911981
Correctly computed : 3841
SNR :  1000.0000007549031
Correctly computed : 3842
SNR :  999.9999998585513
Correctly computed : 3843
SNR :  1000.0000002279542
Correctly computed : 3844
SNR :  1000.0000006119463
Correctly computed : 3845
SNR :  1000.0000005794211
Correctly computed : 

Correctly computed : 3999
SNR :  999.9999999618533
Correctly computed : 4000
SNR :  1000.0000005643866
Correctly computed : 4001
SNR :  999.9999999642941
Correctly computed : 4002
SNR :  1000.0000010495262
Correctly computed : 4003
SNR :  1000.0000003896921
Correctly computed : 4004
SNR :  999.9999998461326
Correctly computed : 4005
SNR :  1000.0000002451094
Correctly computed : 4006
SNR :  1000.0000007924947
Correctly computed : 4007
SNR :  1000.0000006704954
Correctly computed : 4008
SNR :  999.9999998726146
Correctly computed : 4009
SNR :  999.9999997837922
Correctly computed : 4010
SNR :  1000.0000004137878
Correctly computed : 4011
SNR :  999.9999997184728
Correctly computed : 4012
SNR :  1000.0000002792057
Correctly computed : 4013
SNR :  999.9999999153126
Correctly computed : 4014
SNR :  1000.0000005003941
Correctly computed : 4015
SNR :  1000.0000003009545
Correctly computed : 4016
SNR :  1000.0000001337439
Correctly computed : 4017
SNR :  999.9999998645056
Correctly computed :

Correctly computed : 4167
SNR :  1000.0000001113957
Correctly computed : 4168
SNR :  1000.0000000867369
Correctly computed : 4169
SNR :  999.9999996367864
Correctly computed : 4170
SNR :  1000.0000007472155
Correctly computed : 4171
SNR :  999.9999997444583
Correctly computed : 4172
SNR :  999.9999997774538
Correctly computed : 4173
SNR :  1000.0000002294861
Correctly computed : 4174
SNR :  1000.0000004907371
Correctly computed : 4175
SNR :  1000.0000006415652
Correctly computed : 4176
SNR :  1000.0000000042716
Correctly computed : 4177
SNR :  1000.0000003091247
Correctly computed : 4178
SNR :  1000.0000002252644
Correctly computed : 4179
SNR :  1000.0000000338175
Correctly computed : 4180
SNR :  1000.0000003964061
Correctly computed : 4181
SNR :  999.9999995706668
Correctly computed : 4182
SNR :  999.9999995117659
Correctly computed : 4183
SNR :  1000.0000000828642
Correctly computed : 4184
SNR :  1000.000000490224
Correctly computed : 4185
SNR :  1000.0000002291994
Correctly computed

SNR :  999.9999998207439
Correctly computed : 4338
SNR :  1000.0000003277864
Correctly computed : 4339
SNR :  1000.0000000243323
Correctly computed : 4340
SNR :  999.9999999895729
Correctly computed : 4341
SNR :  1000.0000003333619
Correctly computed : 4342
SNR :  999.9999997336781
Correctly computed : 4343
SNR :  999.999999820821
Correctly computed : 4344
SNR :  1000.0000001512877
Correctly computed : 4345
SNR :  1000.0000002117202
Correctly computed : 4346
SNR :  1000.00000025608
Correctly computed : 4347
SNR :  1000.0000002966443
Correctly computed : 4348
SNR :  999.9999997851625
Correctly computed : 4349
SNR :  1000.0000003401781
Correctly computed : 4350
SNR :  1000.0000001335721
Correctly computed : 4351
SNR :  999.9999998383512
Correctly computed : 4352
SNR :  1000.000000308277
Correctly computed : 4353
SNR :  1000.0000000320017
Correctly computed : 4354
SNR :  1000.0000001227129
Correctly computed : 4355
SNR :  1000.0000001165293
Correctly computed : 4356
SNR :  999.99999975204

In [46]:
# Create train and validation set with random SNR

x_train, y_train, prob = extract_features(train_files, y_train, num_ceps)
print('Removed percentage:', prob / len(y_train))
x_val, y_val, prob = extract_features(val_files, y_val, num_ceps)
print('Removed percentage:', prob / len(y_val))


Correctly computed : 0
Correctly computed : 1
Correctly computed : 2
Correctly computed : 3
Correctly computed : 4
Correctly computed : 5
Correctly computed : 6
Correctly computed : 7
Correctly computed : 8
Correctly computed : 9
Correctly computed : 10
Correctly computed : 11
Correctly computed : 12
Correctly computed : 13
Correctly computed : 14
Correctly computed : 15
Correctly computed : 16
Correctly computed : 17
Correctly computed : 18
Correctly computed : 19
Correctly computed : 20
Correctly computed : 21
Correctly computed : 22
Correctly computed : 23
Correctly computed : 24
Correctly computed : 25
Correctly computed : 26
Correctly computed : 27
Correctly computed : 28
Correctly computed : 29
Correctly computed : 30
Correctly computed : 31
Correctly computed : 32
Correctly computed : 33
Correctly computed : 34
Correctly computed : 35
Correctly computed : 36
Correctly computed : 37
Correctly computed : 38
Correctly computed : 39
Correctly computed : 40
Correctly computed : 41
Co

Correctly computed : 374
Correctly computed : 375
Correctly computed : 376
Correctly computed : 377
Correctly computed : 378
Correctly computed : 379
Correctly computed : 380
Correctly computed : 381
Correctly computed : 382
Correctly computed : 383
Correctly computed : 384
Correctly computed : 385
Correctly computed : 386
Correctly computed : 387
Correctly computed : 388
Correctly computed : 389
Correctly computed : 390
Correctly computed : 391
Correctly computed : 392
Correctly computed : 393
Correctly computed : 394
Correctly computed : 395
Correctly computed : 396
Correctly computed : 397
Correctly computed : 398
Correctly computed : 399
Correctly computed : 400
Correctly computed : 401
Correctly computed : 402
Correctly computed : 403
Correctly computed : 404
Correctly computed : 405
Correctly computed : 406
Correctly computed : 407
Correctly computed : 408
Correctly computed : 409
Correctly computed : 410
Correctly computed : 411
Correctly computed : 412
Correctly computed : 413


Correctly computed : 735
Correctly computed : 736
Correctly computed : 737
Correctly computed : 738
Correctly computed : 739
Correctly computed : 740
Correctly computed : 741
Correctly computed : 742
Correctly computed : 743
Correctly computed : 744
Correctly computed : 745
Correctly computed : 746
Correctly computed : 747
Correctly computed : 748
Correctly computed : 749
Correctly computed : 750
Correctly computed : 751
Correctly computed : 752
Correctly computed : 753
Correctly computed : 754
Correctly computed : 755
Correctly computed : 756
Correctly computed : 757
Correctly computed : 758
Correctly computed : 759
Correctly computed : 760
Correctly computed : 761
Correctly computed : 762
Correctly computed : 763
Correctly computed : 764
Correctly computed : 765
Correctly computed : 766
Correctly computed : 767
Correctly computed : 768
Correctly computed : 769
Correctly computed : 770
Correctly computed : 771
Correctly computed : 772
Correctly computed : 773
Correctly computed : 774


Correctly computed : 1090
Correctly computed : 1091
Correctly computed : 1092
Correctly computed : 1093
Correctly computed : 1094
Correctly computed : 1095
Correctly computed : 1096
Correctly computed : 1097
Correctly computed : 1098
Correctly computed : 1099
Correctly computed : 1100
Correctly computed : 1101
Correctly computed : 1102
Correctly computed : 1103
Correctly computed : 1104
Correctly computed : 1105
Correctly computed : 1106
Correctly computed : 1107
Correctly computed : 1108
Correctly computed : 1109
Correctly computed : 1110
Correctly computed : 1111
Correctly computed : 1112
Correctly computed : 1113
Correctly computed : 1114
Correctly computed : 1115
Correctly computed : 1116
Correctly computed : 1117
Correctly computed : 1118
Correctly computed : 1119
Correctly computed : 1120
Correctly computed : 1121
Correctly computed : 1122
Correctly computed : 1123
Correctly computed : 1124
Correctly computed : 1125
Correctly computed : 1126
Correctly computed : 1127
Correctly co

Correctly computed : 1441
Correctly computed : 1442
Correctly computed : 1443
Correctly computed : 1444
Correctly computed : 1445
Correctly computed : 1446
Correctly computed : 1447
Correctly computed : 1448
Correctly computed : 1449
Correctly computed : 1450
Correctly computed : 1451
Correctly computed : 1452
Correctly computed : 1453
Correctly computed : 1454
Correctly computed : 1455
Correctly computed : 1456
Correctly computed : 1457
Correctly computed : 1458
Correctly computed : 1459
Correctly computed : 1460
Correctly computed : 1461
Correctly computed : 1462
Correctly computed : 1463
Correctly computed : 1464
Correctly computed : 1465
Correctly computed : 1466
Correctly computed : 1467
Correctly computed : 1468
Correctly computed : 1469
Correctly computed : 1470
Correctly computed : 1471
Correctly computed : 1472
Correctly computed : 1473
Correctly computed : 1474
Correctly computed : 1475
Correctly computed : 1476
Correctly computed : 1477
Correctly computed : 1478
Correctly co

...

Correctly computed : 2736
Correctly computed : 2737
Correctly computed : 2738
Correctly computed : 2739
Correctly computed : 2740
Correctly computed : 2741
Correctly computed : 2742
Correctly computed : 2743
Correctly computed : 2744
Correctly computed : 2745
Correctly computed : 2746
Correctly computed : 2747
Correctly computed : 2748
Correctly computed : 2749
Correctly computed : 2750
Correctly computed : 2751
Correctly computed : 2752
Correctly computed : 2753
Correctly computed : 2754
Correctly computed : 2755
Correctly computed : 2756
Correctly computed : 2757
Correctly computed : 2758
Correctly computed : 2759
Correctly computed : 2760
Correctly computed : 2761
Correctly computed : 2762
Correctly computed : 2763
Correctly computed : 2764
Correctly computed : 2765
Correctly computed : 2766
Correctly computed : 2767
Correctly computed : 2768
Correctly computed : 2769
Correctly computed : 2770
Correctly computed : 2771
Correctly computed : 2772
Correctly computed : 2773
Correctly co

Correctly computed : 3074
Correctly computed : 3075
Correctly computed : 3076
Correctly computed : 3077
Correctly computed : 3078
Correctly computed : 3079
Correctly computed : 3080
Correctly computed : 3081
Correctly computed : 3082
Correctly computed : 3083
Correctly computed : 3084
Correctly computed : 3085
Correctly computed : 3086
Correctly computed : 3087
Correctly computed : 3088
Correctly computed : 3089
Correctly computed : 3090
Correctly computed : 3091
Correctly computed : 3092
Correctly computed : 3093
Correctly computed : 3094
Correctly computed : 3095
Correctly computed : 3096
Correctly computed : 3097
Correctly computed : 3098
Correctly computed : 3099
Correctly computed : 3100
Correctly computed : 3101
Correctly computed : 3102
Correctly computed : 3103
Correctly computed : 3104
Correctly computed : 3105
Correctly computed : 3106
Correctly computed : 3107
Correctly computed : 3108
Correctly computed : 3109
Correctly computed : 3110
Correctly computed : 3111
Correctly co

Correctly computed : 3415
Correctly computed : 3416
Correctly computed : 3417
Correctly computed : 3418
Correctly computed : 3419
Correctly computed : 3420
Correctly computed : 3421
Correctly computed : 3422
Correctly computed : 3423
Correctly computed : 3424
Correctly computed : 3425
Correctly computed : 3426
Correctly computed : 3427
Correctly computed : 3428
Correctly computed : 3429
Correctly computed : 3430
Correctly computed : 3431
Correctly computed : 3432
Correctly computed : 3433
Correctly computed : 3434
Correctly computed : 3435
Correctly computed : 3436
Correctly computed : 3437
Correctly computed : 3438
Correctly computed : 3439
Correctly computed : 3440
Correctly computed : 3441
Correctly computed : 3442
Correctly computed : 3443
Correctly computed : 3444
Correctly computed : 3445
Correctly computed : 3446
Correctly computed : 3447
Correctly computed : 3448
Correctly computed : 3449
Correctly computed : 3450
Correctly computed : 3451
Correctly computed : 3452
Correctly co

Correctly computed : 3758
Correctly computed : 3759
Correctly computed : 3760
Correctly computed : 3761
Correctly computed : 3762
Correctly computed : 3763
Correctly computed : 3764
Correctly computed : 3765
Correctly computed : 3766
Correctly computed : 3767
Correctly computed : 3768
Correctly computed : 3769
Correctly computed : 3770
Correctly computed : 3771
Correctly computed : 3772
Correctly computed : 3773
Correctly computed : 3774
Correctly computed : 3775
Correctly computed : 3776
Correctly computed : 3777
Correctly computed : 3778
Correctly computed : 3779
Correctly computed : 3780
Correctly computed : 3781
Correctly computed : 3782
Correctly computed : 3783
Correctly computed : 3784
Correctly computed : 3785
Correctly computed : 3786
Correctly computed : 3787
Correctly computed : 3788
Correctly computed : 3789
Correctly computed : 3790
Correctly computed : 3791
Correctly computed : 3792
Correctly computed : 3793
Correctly computed : 3794
Correctly computed : 3795
Correctly co

Correctly computed : 4094
Correctly computed : 4095
Correctly computed : 4096
Correctly computed : 4097
Correctly computed : 4098
Correctly computed : 4099
Correctly computed : 4100
Correctly computed : 4101
Correctly computed : 4102
Correctly computed : 4103
Correctly computed : 4104
Correctly computed : 4105
Correctly computed : 4106
Correctly computed : 4107
Correctly computed : 4108
Correctly computed : 4109
Correctly computed : 4110
Correctly computed : 4111
Correctly computed : 4112
Correctly computed : 4113
Correctly computed : 4114
Correctly computed : 4115
Correctly computed : 4116
Correctly computed : 4117
Correctly computed : 4118
Correctly computed : 4119
Correctly computed : 4120
Correctly computed : 4121
Correctly computed : 4122
Correctly computed : 4123
Correctly computed : 4124
Correctly computed : 4125
Correctly computed : 4126
Correctly computed : 4127
Correctly computed : 4128
Correctly computed : 4129
Correctly computed : 4130
Correctly computed : 4131
Correctly co

In [47]:
# Save MFCCs and their target

feature_sets_file = 'stored_files_targets_SNR.npz'
np.savez(feature_sets_file, 
         x_train=x_train, 
         y_train=y_train, 
         x_val=x_val, 
         y_val= y_val)



In [48]:
# Check if everything worked correctly 

feature_sets = np.load(feature_sets_file)
feature_sets.files

['x_train', 'y_train', 'x_val', 'y_val']